In [1]:
import math
import os

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
labels = '/home/mike/Downloads/Innovation Week 2019/training-labels.csv'
folder = '/home/mike/Downloads/Innovation Week 2019/Train/output_combined2/'
save_folder = '/home/mike/Downloads/Innovation Week 2019/Train/'
test_folder = '/home/mike/Downloads/Innovation Week 2019/Test/'
fill_type = 'crop'  # crop, pad, mix
process_test = False

# # 2015 dataset
# labels = '/home/mike/Downloads/Innovation Week 2019/2015_data/trainLabels.csv'
# folder = '/home/mike/Downloads/Innovation Week 2019/2015_data/Train/train/'
# save_folder = '/home/mike/Downloads/Innovation Week 2019/2015_data/Train/'

# # 2015 dataset - test
# labels = '/home/mike/Downloads/Innovation Week 2019/2015_data/retinopathy_solution.csv'
# folder = '/home/mike/Downloads/Innovation Week 2019/2015_data/Test/test/'
# save_folder = '/home/mike/Downloads/Innovation Week 2019/2015_data/Test/'

# # 2019 aptos dataset
# labels = '/home/mike/Downloads/Innovation Week 2019/aptos2019_data/train.csv'
# folder = '/home/mike/Downloads/Innovation Week 2019/aptos2019_data/Train/train/'
# save_folder = '/home/mike/Downloads/Innovation Week 2019/aptos2019_data/Train/'

In [3]:
label_df = pd.read_csv(labels)
label_df.head()

,Filename,Drscore
0,train_image2.jpeg,0
1,train_image3.jpeg,0
2,train_image4.jpeg,0
3,train_image5.jpeg,0
4,train_image6.jpeg,0


In [4]:
def crop_image(im, amount_to_crop = None):
    w, h = im.size
    if amount_to_crop is None:
        amount_to_crop = abs(w - h)
    if h < w:
        l, r = math.floor(amount_to_crop/2), math.ceil(amount_to_crop/2)
        im = im.crop((l, 0, w-r, h))  # (left, upper, right, lower)
    elif w < h:
        t, b = math.floor(amount_to_crop/2), math.ceil(amount_to_crop/2)
        im = im.crop((0, t, w, h-b))
    return im

def pad_image(im, pad_ratio = 1):
    w, h = im.size
    size_diff = max(w, h) - min(w, h)
    new_size = min(w, h) + size_diff // pad_ratio  # No need to crop first
#     if pad_ratio != 1:
#         amount_to_crop = max(w, h) - new_size
#         im = crop_image(im, amount_to_crop = amount_to_crop)
#         w, h = im.size
    new_im = Image.new('RGB', (new_size, new_size))  # Black by default
    t = math.floor((new_size - h)/2)
    l = math.floor((new_size - w)/2)
    new_im.paste(im, box = (l,t))  # Upper left corner
    return new_im

def preprocess_image(image_path, fill_type = 'crop', desired_size = 224):
    im = Image.open(image_path)
    try:
        if fill_type == 'crop':
            im = crop_image(im)
        elif fill_type == 'pad':
            im = pad_image(im, pad_ratio = 1)
        elif fill_type == 'mix':
            im = pad_image(im, pad_ratio = 2)
    except:
        plt.imshow(im)
        plt.show()
    im = im.resize((desired_size, )*2, resample = Image.LANCZOS)
    return im

In [5]:
def update_progress(progress):
    barLength = 50
    progress = min(max(progress, 0), 1)
    block = int(round(barLength * progress))
    text = "\rPercent: [{0}] {1:.1f}%".format( "#" * block + "-" * (barLength - block), progress * 100)
    print(text, end = '', flush = True)

In [ ]:
N = len(label_df)
print("Files:", N)
x_train = np.empty((N, 224, 224, 3), dtype = np.uint8)
y_train = np.empty((N,), dtype = int)

i = 0
for i in range(N):
    file, group = label_df.loc[i]
    if not (file.endswith('.tif') or file.endswith('.jpeg') or file.endswith('.jpg') or file.endswith('.png')):
        if os.path.exists(os.path.join(folder, file + '.tif')):
            file += '.tif'
        elif os.path.exists(os.path.join(folder, file  + '.jpeg')):
            file += '.jpeg'
        elif os.path.exists(os.path.join(folder, file + '.jpg')):
            file += '.jpg'
        elif os.path.exists(os.path.join(folder, file + '.png')):
            file += '.png'
        else:
            print("Check file extension")
    x_train[i, :, :, :] = preprocess_image(os.path.join(folder, file), fill_type = fill_type)
    y_train[i] = group
    i += 1
    if i % 10 == 0:
        update_progress(i / N)

Files: 14145
Percent: [############################################------] 87.6%

In [ ]:
plt.imshow(x_train[0])

In [ ]:
np.save(os.path.join(save_folder, 'x_{}.npy'.format(fill_type)), x_train)

In [ ]:
y_one_hot = pd.get_dummies(y_train).values
y_train_multi = np.empty(y_one_hot.shape, dtype = int)
y_train_multi[:, 4] = y_one_hot[:, 4]

for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_one_hot[:, i], y_train_multi[:, i+1])

print("Original y_train:", y_one_hot.sum(axis=0))
print("Multilabel version:", y_train_multi.sum(axis=0))

In [ ]:
np.save(os.path.join(save_folder, 'y_{}.npy'.format(fill_type)), y_one_hot)
np.save(os.path.join(save_folder, 'y_multi_{}.npy'.format(fill_type)), y_train_multi)

In [ ]:
########
# Test set
if process_test:
    files = []
    number = []
    for f in sorted(os.listdir(os.path.join(test_folder, 'Test'))):
        if f.endswith('.tif') or f.endswith('.jpeg') or f.endswith('.jpg'):
            files.append(f)
            number.append(int(f.split('.')[0][10:]))
        else:
            print(f)

    number_dict = dict(zip(files, number))
    files = sorted(files, key = number_dict.get)

In [ ]:
if process_test:
    N = len(files)
    print("Files:", N)
    x_test = np.empty((N, 224, 224, 3), dtype = np.uint8)

    i = 0
    for file in files:
        x_test[i, :, :, :] = preprocess_image(os.path.join(test_folder, 'Test', file), fill_type = fill_type)
        i += 1
        if i % 5 == 0:
            update_progress(i / N)
    
    plt.imshow(x_test[-1])

In [ ]:
if process_test:
    np.save(os.path.join(test_folder, 'test_x_{}.npy'.format(fill_type)), x_test)
    pd.DataFrame(files).to_csv(os.path.join(test_folder, 'test_files.csv'), index = None, header = None)